In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import os
import sys

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch import optim
from random import randint 

### para la visualización de los objetos 3D
# TODO: Leer la función y modificarla para imprimir representación sobre GT, (considerar modificar el alpha)
#       Conocer la ubicación de los umbrales para futuras modificaciones

#!curl https://raw.githubusercontent.com/irvingvasquez/nbv-net/master/classification_nbv.py > classification_nbv.py
#sys.path.append('/kaggle/input/nbvsss')
import classification_nbv as cnbv

from utils import net_sample_output_nbv, valida_nbv, entrena_nbv, initialize_weights

ModuleNotFoundError: No module named 'torch'

# Modelo


In [2]:
## Acceso a los datasets
#sys.path.append('/kaggle/input')
#dataset_folder = '/kaggle/input/'
#print(os.listdir('/kaggle/input/interpolated-dataset/archive/training/'))

dataset_folder = 'archive/'

#file_lbl = 'interpolated-dataset/archive/training/dataset_pose_training.npy'
file_lbl = 'validation/dataset_pose_validation.npy'
# batch size
batch_size = 1000

In [3]:
path_input_pose = os.path.join(dataset_folder, file_lbl)
dataset_pose = np.load(path_input_pose)
dataset_pose[2]

array([-0.36613   , -0.0234427 ,  0.159373  , -1.41145649, -1.15631205,
        2.9955435 ])

In [4]:
# Dirección
file_vol = 'validation/dataset_vol_validation_resized_pytorch.npy'

# Lectura de entradas
path_input_vol = os.path.join(dataset_folder, file_vol)
dataset_vol = np.load(path_input_vol)

print("Input data size: \n",dataset_vol.shape)



Input data size: 
 (3034, 29791)


In [5]:
dataset_folder_val = 'archive/validation'
# address
file_vol_val = 'dataset_vol_validation_resized_pytorch.npy'

# load the inputs
path_input_vol = os.path.join(dataset_folder_val, file_vol_val)
dataset_vol = np.load(path_input_vol)

print("Input validation data size: \n",dataset_vol.shape)

Input validation data size: 
 (3034, 29791)


In [6]:
# Carga de los datos
training_dataset = cnbv.NBVClassificationDatasetFull(grid_file= os.path.join(dataset_folder, file_vol), 
                                    nbv_class_file=os.path.join(dataset_folder, file_lbl),
                                    transform=transforms.Compose([
                                    # Reshapes the plain grid
                                    cnbv.ToSO3(),
                                    cnbv.To3DGrid(),
                                    #converts to tensors
                                    cnbv.ToTensor()
                                   ]))
                                                     
validation_dataset = cnbv.NBVClassificationDatasetFull(grid_file= os.path.join(dataset_folder_val, file_vol_val), 
                                    nbv_class_file = os.path.join(dataset_folder_val, 'dataset_pose_validation.npy'),
                                    transform=transforms.Compose([
                                    # Reshapes the plain grid
                                    cnbv.To3DGrid(),
                                    cnbv.ToSO3(),
                                    #converts to tensors
                                    cnbv.ToTensor()
                                    ]))


In [7]:
# Lectura de los datos en batch
train_loader = DataLoader(training_dataset, 
                          batch_size=batch_size,
                          shuffle=True, 
                          num_workers=0)

validation_loader = DataLoader(validation_dataset, 
                          batch_size=batch_size,
                          shuffle=True, 
                          num_workers=0)

In [8]:
print("¿Esta el GPU disponible?:",torch.cuda.is_available())
device = torch.cuda.current_device()

¿Esta el GPU disponible?: True


In [9]:
"""Definir un modelo de MLP 16x2048x512x512x12 basado en cotinuity of rotations"""

class MLP(nn.Module):
    def __init__(self,input = 16, output = 12):
        super(MLP, self).__init__()
        self.input = input
        self.output = output
        self.mlp = nn.Sequential(nn.Linear(self.input,2048),
                                 nn.LeakyReLU(),
                                 nn.Linear(2048,512),
                                 nn.LeakyReLU(),
                                 nn.Linear(512,512),
                                 nn.LeakyReLU(),
                                 nn.Linear(512, self.output),
                                 nn.LeakyReLU())

    
    def forward(self,x):
        
        x = self.mlp(x)
        return x

In [10]:
from torch import linalg as LA
def weighted_loss(output, ground_truth, alpha = 0.5):
   '''a-> Traslation error
    b-> Rotation error'''
   alpha = alpha
   max = 0.4 # 0.396598
   
   #Divide y arregla dimensiones
   l =len(output)
   s = ground_truth[:,:3]
   r = ground_truth[:,3:].reshape([l,3,3])
   s_1 = output[:,:3]
   r_1 = output[:,3:].reshape([l,3,3])
   
   a = LA.norm((s-s_1), dim=-1, ord=2) / max
   b = LA.norm((r-r_1), dim=(-2,-1), ord=2)
   loss = alpha*a + (1-alpha)*b
   return torch.mean(loss)

Si tenemos una esfera de radio no cambiante, es decir, que estamos condicionando el problema a una esfera de radio constante.
Es decir, si conozco la rotación puedo moverme en esa superficie.



In [12]:
num_epochs=1#Numero de epocas

losses = {"entrenamiento":[], "validación":[]}


loss_ant = 10.0 # variable para almacenar el  menor valor de error durante el entrenamiento
torch.manual_seed(5068) # inicialización de semilla
model= MLP().cuda() # Modelo
#Inicialización de pesos con la técnica de Kaiming normal
initialize_weights(model, init = "kaiming_normal", a =None, b= None, mean= None, std= None, const= None, groups= None, gain= None)
criterion=weighted_loss #Metrica error
optimizer=torch.optim.Adam(  #Optimizador
    model.parameters(), 
    lr=1e-3, 
    weight_decay=1e-4)
# ciclo en el número de épocas
for epoch in range(num_epochs):

    print(f"Época {epoch+1}\n-------------------------------")
    loss_entrena = entrena_nbv(train_loader, model, criterion, optimizer, device) #Entrenamieno
    loss_valida = valida_nbv(validation_loader, model, criterion, device)         #Validación
    #============saveb_weights==============
    losses["entrenamiento"].append(loss_entrena.item())
    losses["validación"].append(loss_valida)

    if loss_valida < loss_ant: # si el error en la época actual es menor que el error anterior(el error), almacenas los pesos.
        torch.save(model.state_dict(), 'weights_entrenamiento_MLP.pth')
        loss_ant = loss_valida
        print("loss min: {}".format(loss_ant))

    #================log========================    
    if epoch % 100 == 0:
        loss_value=loss_entrena.cpu().detach().numpy()
        #plotea(train_loader, model)
        print("Numero de epoca:",epoch+1,'Funcion de perdida(entrenamiento):',loss_value)

print('Final\n')    

Época 1
-------------------------------


/media/saulo/datos/Saulo/miniconda3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv3d(


loss min: -5620.5516357421875
Numero de epoca: 1 Funcion de perdida(entrenamiento): -904.9073
Final



# Test

In [2]:
## Acceso a los datasets
#sys.path.append('/kaggle/input')
#dataset_folder = '/kaggle/input/'
#print(os.listdir('/kaggle/input/interpolated-dataset/archive/training/'))

dataset_folder = 'archive/'

#file_lbl = 'interpolated-dataset/archive/training/dataset_pose_training.npy'
file_lbl = 'validation/dataset_pose_validation.npy'
# batch size
batch_size = 1000

In [3]:
path_input_pose = os.path.join(dataset_folder, file_lbl)
dataset_pose = np.load(path_input_pose)
dataset_pose[2]

array([-0.36613   , -0.0234427 ,  0.159373  , -1.41145649, -1.15631205,
        2.9955435 ])

In [4]:
# Dirección
file_vol = 'validation/dataset_vol_validation_resized_pytorch.npy'

# Lectura de entradas
path_input_vol = os.path.join(dataset_folder, file_vol)
dataset_vol = np.load(path_input_vol)

print("Input data size: \n",dataset_vol.shape)

Input data size: 
 (3034, 29791)


In [5]:
dataset_folder_val = 'archive/validation'
# address
file_vol_val = 'dataset_vol_validation_resized_pytorch.npy'

# load the inputs
path_input_vol = os.path.join(dataset_folder_val, file_vol_val)
dataset_vol = np.load(path_input_vol)

print("Input validation data size: \n",dataset_vol.shape)

Input validation data size: 
 (3034, 29791)


In [6]:
# Carga de los datos                                                   
validation_dataset = cnbv.NBVClassificationDatasetFull(grid_file= os.path.join(dataset_folder_val, file_vol_val), 
                                    nbv_class_file = os.path.join(dataset_folder_val, 'dataset_pose_validation.npy'),
                                    transform=transforms.Compose([
                                    # Reshapes the plain grid
                                    cnbv.To3DGrid(),
                                    cnbv.ToSO3(),
                                    #converts to tensors
                                    cnbv.ToTensor()
                                    ]))

In [7]:
# Lectura de los datos en batch
validation_loader = DataLoader(validation_dataset, 
                          batch_size=batch_size,
                          shuffle=False, 
                          num_workers=0)

In [8]:
print("¿Esta el GPU disponible?:",torch.cuda.is_available())
device = torch.cuda.current_device()

¿Esta el GPU disponible?: True


In [9]:
"""Definir un modelo de MLP 16x2048x512x512x12 basado en cotinuity of rotations"""

class MLP(nn.Module):
    def __init__(self,input = 16, output = 12):
        super(MLP, self).__init__()
        self.input = input
        self.output = output
        self.mlp = nn.Sequential(nn.Linear(self.input,2048),
                                 nn.Tanh(),
                                 nn.Linear(2048,512),
                                 nn.Tanh(),
                                 nn.Linear(512,512),
                                 nn.Tanh(),
                                 nn.Linear(512, self.output),
                                 nn.Tanh())

    
    def forward(self,x):
        
        x = self.mlp(x)
        return x

In [10]:
model= MLP().cuda() 
for i in ["xavier_normal", "kaiming_normal"]:
    for j in [1,2,3,4]:
        path_weights = 'stuff/experimento_MLP/weights_entrenamiento_MLP_{}_{}.pth'.format(i,j)
        model.load_state_dict(torch.load(path_weights))
        output, nbv= net_sample_output_nbv(model, validation_loader, device)
        np.save("stuff/experimento_MLP/preds/MLP_{}_{}.npy".format(i,j),output.detach().numpy(), allow_pickle=True)
        del output
        torch.cuda.empty_cache()
np.save("stuff/experimento_MLP/preds/MLP_gt.npy".format(i,j),nbv.detach().numpy(), allow_pickle=True)

/media/saulo/datos/Saulo/miniconda3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv3d(


# Evaluación

In [2]:
from wloss import weighted_loss

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [7]:
gt = torch.from_numpy(np.load('stuff/experimento_MLP/preds/MLP_gt.npy'))
for i in ["xavier_normal", "kaiming_normal"]:
    for j in [1,2,3,4]:
        output = np.load('stuff/experimento_MLP/preds/MLP_{}_{}.npy'.format(i,j))
        output_t = torch.from_numpy(output)
        suma,dif_traslacion,dif_rotacion = weighted_loss(output_t,gt)
        #np.savez('stuff/experimento_MLP/preds/diferencias_rotaciones_{}_{}.npz'.format(i,j), suma_ponderada = suma, traslacion = dif_traslacion, rotacion = dif_rotacion)
        print("El promedio de error con la suma ponderada es: {} de traslación es: {} y de rotación es: {}\n".format(suma,torch.nanmean(dif_traslacion),torch.nanmean(dif_rotacion*180/np.pi)))



El promedio de error con la suma ponderada es: -0.5004704594612122 de traslación es: 0.7488805651664734 y de rotación es: 97.06854248046875

El promedio de error con la suma ponderada es: -0.5018340349197388 de traslación es: 0.7380900382995605 y de rotación es: 96.67076110839844

El promedio de error con la suma ponderada es: -0.4622870981693268 de traslación es: 0.7749289274215698 y de rotación es: 94.12692260742188

El promedio de error con la suma ponderada es: -0.46553629636764526 de traslación es: 0.7706781029701233 y de rotación es: 94.42802429199219

El promedio de error con la suma ponderada es: -0.5264101028442383 de traslación es: 0.7117819786071777 y de rotación es: 98.52960968017578

El promedio de error con la suma ponderada es: -0.5472909808158875 de traslación es: 0.6906380653381348 y de rotación es: 100.11519622802734

El promedio de error con la suma ponderada es: -0.502632737159729 de traslación es: 0.7292459607124329 y de rotación es: 96.28189849853516

El promedio 

In [5]:
torch.nonzero(torch.isnan(dif_rotacion.view(-1))).size()

torch.Size([238, 1])

In [20]:
gt[0,3:]

tensor([-5.2573e-01,  8.5065e-01,  7.0377e-17, -6.4384e-17,  4.2942e-17,
        -1.0000e+00, -8.5065e-01, -5.2573e-01,  3.2192e-17])

In [36]:
output_t[0,3:]

tensor([ 0.2028,  0.1101, -0.0536,  0.0370,  0.0572, -0.9722, -0.0321,  0.1899,
        -0.0336])

In [27]:
M_biprimo = gt[0,3:].reshape([3,3])*torch.linalg.inv(output_t[0,3:].reshape([3,3]))

In [28]:
M_biprimo

tensor([[-2.3923e+00, -1.3742e-01, -1.8230e-16],
        [-5.2002e-17, -9.1291e-18, -4.8622e+00],
        [-1.8779e-01,  5.5055e-01,  6.0262e-18]])

In [45]:
g_error = (M_biprimo[0,0]+M_biprimo[1,1]+M_biprimo[2,2]-1)

In [46]:
g_error

tensor(-3.3923)

In [52]:
torch.arccos(torch.tensor(-.5))

tensor(2.0944)

In [42]:
1.9855+1*2-

3.9855

In [13]:
dif_traslacion.max()

tensor(1.8729)

In [4]:
dif_rotacion

tensor([   nan,    nan, 1.9855, 1.1825, 1.5351,    nan, 1.6849, 0.8658, 1.5997,
        2.0868, 1.6266, 2.0027,    nan, 1.5923,    nan, 2.2434, 2.0404,    nan,
        1.7409, 2.2349,    nan,    nan, 1.2775, 0.9618, 1.1289, 1.8349, 2.0823,
        2.1541, 1.9845,    nan, 1.9936, 1.8484, 1.0427,    nan, 1.6846,    nan,
        1.3276, 1.2346, 2.7687, 1.9244, 2.3117, 2.7389, 2.1193, 1.8519, 1.3239,
        2.1015,    nan, 2.0722, 0.9698, 1.1932, 1.9555, 1.0977, 2.0288,    nan,
           nan, 1.8160, 1.4122, 2.0729, 1.5193, 1.1910, 1.5861, 1.2319,    nan,
        1.9685, 2.7848, 2.0770, 1.8136, 1.7212, 1.8927, 1.0650,    nan, 1.0016,
        1.0952, 1.6330,    nan, 1.7987, 2.5167,    nan, 1.1990, 1.8820, 1.8761,
        2.6713, 1.6947, 1.7383, 1.4352, 2.0193, 1.7250,    nan, 1.9571, 1.1294,
        1.7687, 2.9575, 1.8076, 1.2474, 1.9886,    nan, 1.8275,    nan,    nan,
           nan,    nan,    nan, 0.9740, 2.0312, 1.4973, 0.1502, 0.4292, 1.9171,
        0.6852, 1.6590, 1.6479, 2.0803, 

In [5]:
dif_rotacion*180/np.pi

tensor([     nan,      nan, 113.7581,  67.7534,  87.9574,      nan,  96.5356,
         49.6054,  91.6569, 119.5670,  93.1966, 114.7472,      nan,  91.2317,
             nan, 128.5368, 116.9089,      nan,  99.7445, 128.0478,      nan,
             nan,  73.1933,  55.1062,  64.6789, 105.1331, 119.3066, 123.4233,
        113.7058,      nan, 114.2222, 105.9027,  59.7415,      nan,  96.5179,
             nan,  76.0669,  70.7353, 158.6364, 110.2613, 132.4500, 156.9274,
        121.4257, 106.1061,  75.8529, 120.4056,      nan, 118.7304,  55.5627,
         68.3648, 112.0424,  62.8928, 116.2398,      nan,      nan, 104.0471,
         80.9113, 118.7672,  87.0493,  68.2372,  90.8746,  70.5836,      nan,
        112.7893, 159.5593, 119.0011, 103.9122,  98.6167, 108.4417,  61.0208,
             nan,  57.3849,  62.7486,  93.5651,      nan, 103.0580, 144.1972,
             nan,  68.6963, 107.8300, 107.4953, 153.0553,  97.1018,  99.5971,
         82.2315, 115.6954,  98.8359,      nan, 112.1337,  64.70

In [6]:
dif_rotacion*180/np.pi

tensor([     nan,      nan, 113.7581,  67.7534,  87.9574,      nan,  96.5356,
         49.6054,  91.6569, 119.5670,  93.1966, 114.7472,      nan,  91.2317,
             nan, 128.5368, 116.9089,      nan,  99.7445, 128.0478,      nan,
             nan,  73.1933,  55.1062,  64.6789, 105.1331, 119.3066, 123.4233,
        113.7058,      nan, 114.2222, 105.9027,  59.7415,      nan,  96.5179,
             nan,  76.0669,  70.7353, 158.6364, 110.2613, 132.4500, 156.9274,
        121.4257, 106.1061,  75.8529, 120.4056,      nan, 118.7304,  55.5627,
         68.3648, 112.0424,  62.8928, 116.2398,      nan,      nan, 104.0471,
         80.9113, 118.7672,  87.0493,  68.2372,  90.8746,  70.5836,      nan,
        112.7893, 159.5592, 119.0011, 103.9122,  98.6167, 108.4417,  61.0209,
             nan,  57.3849,  62.7486,  93.5651,      nan, 103.0579, 144.1972,
             nan,  68.6963, 107.8300, 107.4953, 153.0553,  97.1018,  99.5971,
         82.2315, 115.6954,  98.8359,      nan, 112.1337,  64.70

# Pruebas
